This script is used to train the first YOLOv5 mode on the given 1,800 images of SnakeCLEF 2021.

# Get YOLOv5

In [ ]:
! git clone https://github.com/ultralytics/yolov5  # clone
% cd yolov5
% pip install -qr requirements.txt  # install
% cd yolov5_projektarbeit

# Setup

In [ ]:
import torch
from psutil import virtual_memory
import shutil
import os
import gc
from matplotlib import pyplot as plt
import numpy as np
import cv2
from matplotlib.pyplot import figure

In [ ]:
# check YOLOv5
%cd yolov5
from yolov5 import utils
display = utils.notebook_init()  # checks

In [ ]:
# check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# (optional) Check RAM
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Move YAML files to YOLOv5 installation
### Model meta information
Here we tell YOLOv5 where to find the data as well as how many classes there are and what they are called. Are created manually.

<code>
    path: ../images # dataset root dir
    train: [train/images, train/labels]  
    val: [val/images, val/labels]

    \# Classes
    nc: 1  # number of classes
    names: ['snake']  # class names
</code>

## Model settings
Here we only adjust the number of classes (nc) and copy&paste the predefined model architecture. For the YOLOv5 modelv5l this looks like this:
<code>
    \# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

    \# Parameters
    nc: 1  \# number of classes
    depth_multiple: 1.0  \# model depth multiple
    width_multiple: 1.0  \# layer channel multiple
    anchors:
      - [10,13, 16,30, 33,23]  \# P3/8
      - [30,61, 62,45, 59,119]  # P4/16
      - [116,90, 156,198, 373,326]  \# P5/32

    \# YOLOv5 v6.0 backbone
    backbone:
      \# [from, number, module, args]
      [[-1, 1, Conv, [64, 6, 2, 2]],  \# 0-P1/2
       [-1, 1, Conv, [128, 3, 2]],  \# 1-P2/4
       [-1, 3, C3, [128]],
       [-1, 1, Conv, [256, 3, 2]],  \# 3-P3/8
       [-1, 6, C3, [256]],
       [-1, 1, Conv, [512, 3, 2]],  \# 5-P4/16
       [-1, 9, C3, [512]],
       [-1, 1, Conv, [1024, 3, 2]],  \# 7-P5/32
       [-1, 3, C3, [1024]],
       [-1, 1, SPPF, [1024, 5]],  \# 9
      ]

    \# YOLOv5 v6.0 head
    head:
      [[-1, 1, Conv, [512, 1, 1]],
       [-1, 1, nn.Upsample, [None, 2, 'nearest']],
       [[-1, 6], 1, Concat, [1]],  \# cat backbone P4
       [-1, 3, C3, [512, False]],  \# 13

       [-1, 1, Conv, [256, 1, 1]],
       [-1, 1, nn.Upsample, [None, 2, 'nearest']],
       [[-1, 4], 1, Concat, [1]],  \# cat backbone P3
       [-1, 3, C3, [256, False]],  \# 17 (P3/8-small)

       [-1, 1, Conv, [256, 3, 2]],
       [[-1, 14], 1, Concat, [1]],  \# cat head P4
       [-1, 3, C3, [512, False]],  \# 20 (P4/16-medium)

       [-1, 1, Conv, [512, 3, 2]],
       [[-1, 10], 1, Concat, [1]],  \# cat head P5
       [-1, 3, C3, [1024, False]],  \# 23 (P5/32-large)

       [[17, 20, 23], 1, Detect, [nc, anchors]],  \# Detect(P3, P4, P5)
      ]
</code>

In [ ]:
# Model settings
path_to_model = 'OWN_PATH/yolov5_yaml/model_settings/model_settings' # Manuelly creatd model definition.
target_model = 'yolov5l_snake.yaml' # select your model 
shutil.copy('{}/{}'.format(path_to_model,target_model),'OWN_PATH/yolov5/models') # Paste to YOLOv5 installation

# Model meta infos
path_to_meta = 'OWN_PATH/yolov5_yaml/snake_detection.yaml' # Manually created YAML

shutil.copy(path_to_meta,'OWN_PATH/yolov5/data') # Past to YOLOv5 installation


# Check data
Check whether the data from script 01 has been sorted correctly.

In [ ]:
print('TRAIN')
print('Images',len(os.listdir('OWN_PATH/images/train/images')))
print('Labels',len(os.listdir('OWN_PATH/images/train/labels')))

print('VAL')
print('Images',len(os.listdir('OWN_PATH/images/val/images')))
print('Labels',len(os.listdir('OWN_PATH/images/val/labels')))

# Train a YOLOv5 model
A brief overview of the parameters used:
<code>
    --img 640 # Max. image input size (640px)
    --batch 14 # Number of batch
    --cfg yolov5/models/yolov5l_snake.yaml # Pass the model structure to YOLOv5
    --epochs 200 # Number of epochs used for training
    --data yolov5/data/snake_detection.yaml # Tell YOLOv5 where it can find the corresponding data
    --cache # YOLOv5 uses RAM to speed up the calculation
    --weights yolov5l.pt # Specify the weights that YOLOv5 should use
    </code>
Various graphs on metrics of the training can be found in the folder given by YOLOv5. It is worth looking at the created files.

In [ ]:
!python yolov5/train.py --img 640 --batch 14 --cfg yolov5/models/yolov5l_snake.yaml --epochs 200 --data yolov5/data/snake_detection.yaml --cache --weights yolov5l.pt 


# Save YOLOv5 run


In [ ]:
run = 'exp' 
model_name = 'OUR_NAME_FOR_RUN' 

shutil.copytree(path_to_run,'OWN_PATH/saved_runs/exp_{}'.format(model_name))